In [ ]:
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
emotion_df=pd.read_csv('/content/text.csv')

In [ ]:
emotion_df.head()

In [ ]:
violence_df=pd.read_csv('/content/Train.csv')

In [ ]:
violence_df.head()

In [ ]:
hate_df=pd.read_csv('/content/labeled_data.csv')

In [ ]:
hate_df.head()

In [ ]:
#data preprocessing

emotion_df.drop(columns=['Unnamed: 0'],inplace=True)
violence_df.drop(columns=['Tweet_ID'] ,inplace=True)
hate_df=hate_df[['tweet', 'class']]

In [ ]:
emotion_df.head()


In [ ]:
violence_df.head()


In [ ]:
hate_df.head()

In [ ]:
emotion_df.columns, violence_df.columns, hate_df.columns

In [ ]:
#renaming columns
violence_df.rename(columns={'tweet':'text', 'type':'label'},inplace=True)
hate_df.rename(columns={'tweet':'text', 'class':'label'},inplace=True)

In [ ]:
emotion_df.columns, violence_df.columns, hate_df.columns

In [ ]:
# null values
emotion_df.isnull().sum() , violence_df.isnull().sum(), hate_df.isnull().sum()

In [ ]:
emotion_df.shape, violence_df.shape, hate_df.shape

In [ ]:
#extarcting 12000 from each dataset
emotion_df['label'].value_counts()

In [ ]:
e_df=pd.DataFrame()
for i in range(6):
  subset= emotion_df[emotion_df['label']==i].sample(n=2000, random_state=42 )
  e_df=pd.concat([e_df, subset])

In [ ]:
e_df.shape

In [ ]:
emotion_df=e_df.copy()

In [ ]:
emotion_df['label'].value_counts()

In [ ]:
violence_df['label'].value_counts()

In [ ]:
sexual_vioelnce=violence_df[violence_df['label']=='sexual_violence'].sample(n=4998, random_state=42)
violence_df=violence_df[violence_df['label']!='sexual_violence']

In [ ]:
violence_df.shape

In [ ]:
violence_df=pd.concat([violence_df, sexual_vioelnce], axis=0)

In [ ]:
violence_df.shape

In [ ]:
hate_df['label'].value_counts()


In [ ]:
offensive_speech= hate_df[hate_df['label']== 1].sample(n=6407, random_state=42)
hate_df= hate_df[hate_df['label']!=1]

In [ ]:
hate_df.shape

In [ ]:
hate_df=pd.concat([offensive_speech, hate_df], axis=0)

In [ ]:
hate_df.shape

In [ ]:
emotion_df.shape, violence_df.shape, hate_df.shape

In [ ]:
#reset indexes
emotion_df.reset_index(drop=True, inplace=True)
violence_df.reset_index(drop=True, inplace=True)
hate_df.reset_index(drop=True, inplace=True)


In [ ]:
emotion_df.head()

//label encoding


In [ ]:
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
label_encoder=LabelEncoder()
violence_df['label']=label_encoder.fit_transform(violence_df['label'])
violence_df.head()

In [ ]:
violence_df['label'].unique()

//stopwords removal


In [ ]:
import nltk
from nltk.corpus import stopwords


In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')#puntuation

In [ ]:
stop_words=set(stopwords.words('english'))

In [ ]:
len(stop_words)

In [ ]:
nltk.download('punkt_tab')

In [ ]:
def remove_stopwords(text):
  all_words=nltk.word_tokenize(text)
  filtered_words=[word for word in all_words if word.lower() not in stop_words]
  return ' '.join(filtered_words)

emotion_df['text']=emotion_df['text'].apply(remove_stopwords)
violence_df['text']=violence_df['text'].apply(remove_stopwords)
hate_df['text']=hate_df['text'].apply(remove_stopwords)

//tokenization & padding




In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(pd.concat([emotion_df['text'], violence_df['text'], hate_df['text']]))

In [ ]:
emotion_sequences=tokenizer.texts_to_sequences(emotion_df['text'])
violence_sequences=tokenizer.texts_to_sequences(violence_df['text'])
hate_sequences=tokenizer.texts_to_sequences(hate_df['text'])

In [ ]:
emotion_df['text'].iloc[2]

In [ ]:
emotion_sequences[3:6]

In [ ]:
max_length=50
emotion_padded = pad_sequences(emotion_sequences, maxlen=max_length, padding='post')
violence_padded = pad_sequences(violence_sequences, maxlen=max_length, padding='post')
hate_padded = pad_sequences(hate_sequences, maxlen=max_length, padding='post')

In [ ]:
import numpy as np


In [ ]:
# //generating labels in numpy array format
emotion_labels=np.array(emotion_df['label'])
violence_labels=np.array(violence_df['label'])
hate_labels=np.array(hate_df['label'])

//MODEL DEFINITION

In [ ]:
emotion_input=emotion_padded
violence_input=violence_padded
hate_input=hate_padded

In [ ]:
from tensorflow import keras

In [ ]:
##definig multiple input layer
emotion_input_layer=keras.layers.Input(shape=(max_length,),name='emotion_input')
violence_input_layer=keras.layers.Input(shape=(max_length,),name='violence_input')
hate_input_layer=keras.layers.Input(shape=(max_length,),name='hate_input')

In [ ]:
##use shared embedding layer
embedding_layer=keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128)

In [ ]:
#apply embedding layer to each input
emotiom_embedding_layer=embedding_layer(emotion_input_layer)
violence_embedding_layer=embedding_layer(violence_input_layer)
hate_embedding_layer=embedding_layer(hate_input_layer)



In [ ]:
#shared lstm layer
shared_lstm=keras.layers.LSTM(64,return_sequences=True)

In [ ]:
emotion_lstm=shared_lstm(emotiom_embedding_layer)
violence_lstm=shared_lstm(violence_embedding_layer)
hate_lstm=shared_lstm(hate_embedding_layer)

In [ ]:
#shared global average pooling layyer and dropout layer
shared_pooling= keras.layers.GlobalAveragePooling1D()
shared_dropout=keras.layers.Dropout(0.5)

In [ ]:
emotion_lstm=shared_pooling(emotion_lstm)
violence_lstm=shared_pooling(violence_lstm)
hate_lstm=shared_pooling(hate_lstm)

In [ ]:
emotion_features=shared_dropout(emotion_lstm)
violence_features=shared_dropout(violence_lstm)
hate_features=shared_dropout(hate_lstm)

In [ ]:
#output layers
emotion_output=keras.layers.Dense(6, activation='softmax', name='emotion_output')(emotion_features)
violence_output=keras.layers.Dense(5, activation='softmax', name='violence_output')(violence_features)
hate_output=keras.layers.Dense(3, activation='softmax', name='hate_output')(hate_features)

In [ ]:
#compile the model with multiple inputs and outputs
model=keras.Model(inputs=[emotion_input_layer, violence_input_layer, hate_input_layer], outputs=[emotion_output, violence_output, hate_output])
model.compile(optimizer='adam', loss={'emotion_output': 'sparse_categorical_crossentropy', 'violence_output':'sparse_categorical_crossentropy','hate_output':'sparse_categorical_crossentropy'}, metrics={'emotion_output':'accuracy', 'violence_output':'accuracy', 'hate_output':'accuracy'})
model.summary()

In [ ]:
#trainig model
model.fit(x={'emotion_input': emotion_input,
             'violence_input': violence_input,
             'hate_input': hate_input},
          y={'emotion_output': emotion_labels,
             'violence_output': violence_labels,
             'hate_output': hate_labels},
          epochs=10,
          batch_size=4
             )

In [ ]:
prediction= model.predict({'emotion_input': emotion_input,
             'violence_input': violence_input,
             'hate_input': hate_input})

In [ ]:
prediction

In [ ]:
emotion_pred=np.argmax(prediction[0], axis=1)[0]
violence_pred=np.argmax(prediction[1], axis=1)[0]
hate_pred=np.argmax(prediction[2], axis=1)[0]

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
def plot_cm(true, title, labels):
  cf_matrix=confusion_matrix(true, pred, normalize='true')
  plt.figure(figsize=(7,6))
  sns.heatmap(cf_matrix, annot=True, cmap='Blues', xticklabels=labels, yticklabels=labels)
  plt.title(title)
  plt.xlabel('Predicted')
  plt.ylabel('actual')

emotion_labels_text=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
violence_labels_text=['sexual_violence', 'physical_violence', 'emotional_violence', 'Harmful_traditional_practice', 'economic_violence']
hate_labels_text=['offensive speech', 'Neither', 'Hate Speech']


In [ ]:
# plot_cm(emotion_labels, emotion_pred, 'Confusion Matrix for Emotion')
# plot_cm(violence_labels, violence_pred, 'Confusion Matrix for Violence')
# plot_cm(hate_labels, hate_pred, 'Confusion Matrix for Hate')

//manual testing


In [ ]:
from os import remove
def classify_text(input_text):
  input_text_cleaned=remove_stopwords(input_text)
  input_sequence=tokenizer.texts_to_sequences([input_text_cleaned])
  input_padded=pad_sequences(input_sequence, maxlen=max_length, padding='post')

  predictions=model.predict({'emotion_input': input_padded,
                             'violence_input': input_padded,
                             'hate_input': input_padded})

  emotion_pred=np.argmax(predictions[0], axis=1)[0]
  violence_pred=np.argmax(predictions[1], axis=1)[0]
  hate_pred=np.argmax(predictions[2], axis=1)[0]

  major_labels=['Emotion', 'Violence', 'Hate']
  major_label_index= np.argmax([np.max(predictions[0]), np.max(predictions[1]), np.max(predictions[2])])
  major_label=major_labels[major_label_index]

  #datermining sublabels
  emotion_labels_text=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
  violence_labels_text=['sexual_violence', 'physical_violence', 'emotional_violence', 'Harmful_traditional_practice', 'economic_violence']
  hate_labels_text=['offensive speech', 'Neither', 'Hate Speech']

  if major_label=='Emotion':
    sub_label=emotion_labels_text[emotion_pred]
  elif major_label=='Violence':
    sub_label=violence_labels_text[violence_pred]
  else:
    sub_label= hate_labels_text[hate_pred]

  return major_label, sub_label




In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
#define a text widget and a placeholder
input_text_widget=widgets.Text(
    description='Input text',
    placeholder='Enter your text here'
)

In [ ]:
button=widgets.Button(description='Classify')

In [ ]:
output=widgets.Output()


In [ ]:
def on_button_click(b):
  with output:
    output.clear_output()
    input_text=input_text_widget.value
    major_label, sub_label=classify_text(input_text)
    print(f'Major Label: {major_label}')
    print(f'Sub Label: {sub_label}')

button.on_click(on_button_click)

In [ ]:
display(input_text_widget, button, output)

In [ ]:
display(input_text_widget, button, output)